In [1]:
import argparse
import numpy as np
from scipy.io import wavfile

In [2]:
from audio.processor import WavProcessor, format_predictions

In [3]:
import csv
import os
import tensorflow as tf

from audio import params
from audio.utils import vggish, youtube8m

In [4]:
wav_file = "wav_files/07063137_car.wav"
sr, data = wavfile.read(wav_file)

C:\Users\myrna\Anaconda3\lib\site-packages\scipy\io\wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [5]:
proc = WavProcessor()

INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt
INFO:tensorflow:Restoring parameters from models/youtube_model.ckpt


In [6]:
# Normalize (between -1 and 1)
samples = data / 32768.0

In [7]:
# Converts audio waveform into an array of examples for VGGish
"""Returns:
      3-D np.array of shape [num_examples, num_frames, num_bands] which represents
      a sequence of examples, each of which contains a patch of log mel
      spectrogram, covering num_frames frames of audio and num_bands mel frequency
      bands, where the frame length is params.STFT_HOP_LENGTH_SECONDS"""
examples_batch = vggish.input.waveform_to_examples(samples, sr)

In [8]:
examples_batch.shape

(31, 96, 64)

In [9]:
features = proc._get_features(examples_batch)

In [11]:
features.shape

(31, 128)

In [14]:
predictions = proc._process_features(features)

In [23]:
predictions = proc._filter_predictions(predictions)

In [15]:
predictions

array([[  3.99287819e-05,   2.19806175e-17,   8.01216829e-16,
          3.15704298e-08,   3.06891252e-14,   3.23250768e-17,
          1.01651581e-21,   3.16726556e-09,   4.50789267e-11,
          2.62651463e-11,   1.64979562e-13,   1.78366434e-21,
          7.87573078e-13,   1.20186951e-11,   1.35279557e-11,
          2.58460140e-16,   1.62613163e-15,   4.76833692e-19,
          2.41697367e-19,   5.28037764e-14,   5.85652081e-20,
          5.24910571e-22,   5.40609665e-21,   2.31306715e-17,
          1.10571682e-11,   4.40971956e-20,   1.09819674e-15,
          1.68454704e-15,   5.16530628e-23,   7.73199376e-13,
          2.66837401e-16,   5.80548441e-18,   2.31589740e-16,
          1.76154663e-17,   9.31880999e-19,   2.31653482e-20,
          2.34175978e-20,   7.32949852e-19,   3.11580355e-16,
          7.77414842e-20,   7.49254516e-25,   1.82052247e-12,
          3.33205848e-25,   3.94250890e-17,   2.54685162e-10,
          2.56169316e-25,   3.68609527e-12,   5.85982582e-21,
        

In [29]:
len(predictions)

1

In [39]:
predictions.size

527